---
title: Interactive Visualization Notebook of Key Results

---

# Interactive Visualization Notebook of Key Results

This notebook creates interactive visualizations for exploring groundwater recharge patterns, river networks, and subsurface characteristics in California's Central Valley.

## Setup and Imports

**Key Libraries:**

- **`xarray` & `rioxarray`**: Handle multi-dimensional raster data with geospatial awareness
- **`geopandas`**: Manage vector geospatial data (rivers, basins, points)
- **`ipyleaflet`**: Create interactive Leaflet.js maps in Jupyter
- **`ipywidgets`**: Build interactive UI controls (dropdowns, sliders)
- **`numpy`**: Numerical operations
- **`pandas`**: Tabular data manipulation
- **`shapely`**: Geometric operations

In [1]:
# -------------------------------
# Imports
# -------------------------------
import xarray as xr
import geopandas as gpd
from ipyleaflet import (
    ImageOverlay,
    GeoJSON,
    TileLayer,
)

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
from utils import (
    leaflet_bounds,
    scalar_to_base64_image,
    find_intersections,
    combine_rivers_gdf,
    format_valley_name,
    DualMapController,
)

## Configuration

Define file paths, coordinate system, and visualization parameters.

**Directory Structure:**
- `data_download/`: Raw data files (shapefiles, CSV, rasters)
- `data_intermediate/`: Processed/consolidated datasets (Zarr)

**Coordinate Reference System:**
- **EPSG:4326** (WGS84 lat/lon) - Standard for web mapping
- All data will be reprojected to this CRS for consistency

**Map Settings:**
- Center coordinates: Central Valley, California
- Zoom level: 7 (regional view)
- Map dimensions: 500×800px

In [2]:
# -------------------------------
# Data Paths
# -------------------------------
export_dir = "./data_export"

# Vector data (geospatial boundaries and features)
vector_rivers = f"{export_dir}/rivers.geojson"
vector_subbasin = f"{export_dir}/subbasin.geojson"
vector_brackish = f"{export_dir}/brackish.geojson"

# Raster/gridded data
metric_zarr_path = f"{export_dir}/consolidated_metric_output.zarr"

# -------------------------------
# Map Configuration
# -------------------------------

center = [37.66335291403956, -120.69523554193438]  # Central Valley center
zoom = 7  # Regional zoom level
basemap = None  # Use custom tile layers instead of default basemap
map_width = "500px"
map_height = "800px"

colors = [
    (0, 0.5, 0),  # Green
    (1, 1, 1),
]  # White
positions = [0, 1]
green_r_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

colors = [
    (1, 1, 1),  # Green
    (0, 0.5, 0),
]  # White
positions = [0, 1]
green_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

# Extract RGB values from the predefined RdBu colormap
cmap = plt.get_cmap("RdBu")

# Create a list of colors by sampling the colormap
colors = [cmap(i) for i in range(cmap.N)]

# Create the custom colormap
custom_rdbu = LinearSegmentedColormap.from_list("custom_rdbu", np.array(colors)[20:-20][::-1])

param_dict = {
    "fraction_coarse": {
        "vmin": 0,
        "vmax": 100,
        "short_label": "FCD",
        "label": "Fraction of coarse-dominated sediment (%)",
        "cmap": custom_rdbu,
        "log_scale": False,
    },
    "path_length_norm": {
        "vmin": 1,
        "vmax": 1000,
        "short_label": "Normalized distance",
        "label": "Normalized path length",
        "cmap": green_r_map,
        "log_scale": True,
    },
    "path_to_no_flow": {
        "vmin": 0,
        "vmax": 20,
        "short_label": "Depth to no flow",
        "label": "Depth to shallowest no-flow unit\n or base surface (m)",
        "cmap": green_map,
        "log_scale": False,
    },
}

## Data Loading and Preprocessing

Load all datasets and prepare them for visualization by:
1. Reprojecting to common CRS (EPSG:4326)
2. Processing geometries for performance
3. Computing derived features

### Processing Steps

**Vector Data (Rivers & Basins):**
- Load GeoJSON files with `geopandas`
- Dissolve river fragments with same name into single geometries
- Simplify geometries (~100m tolerance) for faster rendering
- Find river-basin boundary intersections (exit/inflow points)

**Point Data (Resistivity):**
- Load CSV with UTM coordinates
- Convert to GeoDataFrame with proper CRS (EPSG:3310 → 4326)
- Subsample to 10,000 points for performance

**Raster Data (Metrics):**
- Open Zarr dataset with `xarray`
- Reorder and sort dimensions for proper display
- Reproject from CA Albers (EPSG:3310) to WGS84 (EPSG:4326)

**Performance Optimizations:**
- Geometry simplification reduces vertex count
- Point subsampling reduces render time
- Lazy loading with Zarr for large datasets

In [3]:
# -------------------------------
# Load Vector Layers
# -------------------------------
brackish = gpd.read_file(vector_brackish)
rivers = gpd.read_file(vector_rivers)
# Combine river segments that share the same name
rivers = combine_rivers_gdf(rivers)
# TODO: QC this - some rivers have gaps and potential duplicate names

subbasins = gpd.read_file(vector_subbasin)
# Apply formatting to subbasin names
subbasins["Basin_Su_1"] = subbasins["Basin_Su_1"].apply(format_valley_name)

# Find points where rivers exit the basin boundary
river_intersections = find_intersections(rivers, subbasins)

# -------------------------------
# Load Raster Data (Consolidated Metrics)
# -------------------------------
ds = xr.open_zarr(metric_zarr_path)
ds = ds.drop_vars("spatial_ref")

## Layer Creation

Create all map layers for interactive visualization. Each layer can be toggled on/off independently.

### Layer Types

**Tile Layers (Basemaps):**
- `ipyleaflet.TileLayer` fetches map tiles from remote servers
- Ocean layer: Shows water features and bathymetry
- Hillshade layer: Provides terrain context with shaded relief

**Vector Layers:**
- `ipyleaflet.GeoJSON` renders vector features client-side
- Rivers: Blue lines showing drainage network
- Subbasins: Black outlines defining hydrologic boundaries
- Intersections: Orange markers at river exit points

**Point Visualization:**
- **GeoJSON points**: Individual markers (very small radius for dense data)
- **Heatmap**: Density visualization using kernel density estimation
  - `radius`: Influence distance of each point (pixels)
  - `blur`: Smoothing factor
  - `gradient`: Color scale from low to high density

**Use Cases:**
- Points: Precise locations
- Heatmaps: Spatial patterns and sampling density

In [4]:
# -------------------------------
# Basemap Tile Layers
# -------------------------------
# Ocean basemap with bathymetry from ESRI ArcGIS Online
l_ocean = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}",
    name="Ocean/Water",
    opacity=1.0,
)

# Hillshade elevation for terrain context
l_elevation = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}",
    name="Elevation",
    opacity=1.0,
)

# -------------------------------
# Vector Layers (Lines & Polygons)
# -------------------------------
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={"color": "blue", "weight": 1, "opacity": 0.7},
    name="Rivers",
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={"color": "black", "weight": 1, "fill": False},
    name="Subbasins",
)

l_brackish = GeoJSON(
    data=brackish.__geo_interface__,
    style={
        "color": "black",
        "weight": 1,
        "fillColor": "gray",
        "fillOpacity": 1.0,
    },
    name="Brackish Areas",
)

# River exit/inflow points at basin boundary
intersections_layer = GeoJSON(
    data=river_intersections.__geo_interface__,
    point_style={
        "radius": 6,
        "color": "orange",
        "fillColor": "orange",
        "fillOpacity": 0.5,
        "width": 2,
    },
    name="River Exits",
)

# -------------------------------
# Create Raster Overlays
# -------------------------------
# Left map: Fraction Coarse Dominated (static reference)
fcd_params = param_dict["fraction_coarse"]
fcd_ave = ds["fraction_coarse"]
fcd_layer = ImageOverlay(
    url=scalar_to_base64_image(
        fcd_ave,
        cmap=fcd_params["cmap"],
        vmin=fcd_params["vmin"],
        vmax=fcd_params["vmax"],
        log_scale=fcd_params["log_scale"],
    ),
    bounds=leaflet_bounds(fcd_ave),
    opacity=1.0,
    name="Raster Dataset",  # Generic name for layer control
)

## Figures 2-7: Dual Synchronized Maps

Side-by-side interactive maps with synchronized views for comparative analysis.

### Architecture

**DualMapController Class:**
A custom controller that manages two linked maps with shared navigation and independent data layers.

**Key Features:**
1. **View Synchronization**: Zoom and pan are linked via `ipywidgets.jslink()`
2. **Independent Layers**: Each map displays different data for comparison
3. **Interactive Highlighting**: Click features to highlight across both maps
4. **Dynamic Data Updates**: Switch datasets and thresholds in real-time

### Map Layout

**Left Map (m1):**
- Fraction Coarse Dominated (FCD) raster overlay
- Subbasin selector with zoom-to-region
- River exit point markers (clickable)
- Shows static reference data

**Right Map (m2):**
- Switchable metric datasets (path length, recharge, etc.)
- Threshold slider for FCD filtering
- Same base layers and markers
- Shows dynamic analysis results

### Interactive Workflows

**Subbasin Selection:**
1. Choose from dropdown → map zooms to bounds
2. Subbasin outline highlighted with orange dashed border
3. Highlight persists until new selection
4. "All Regions" returns to default view

**River Selection:**
1. Click orange marker at river exit point
2. Full river geometry highlights in bright blue on both maps
3. Popup shows river name
4. Multiple river highlights overlay (compare drainage patterns)

**Dataset Exploration:**
1. Select metric from dropdown (right map only)
2. Adjust FCD threshold with slider
3. Right map updates raster overlay
4. Left map unchanged for reference comparison

### Technical Implementation

**Synchronization:**
```python
jslink((map1, 'center'), (map2, 'center'))
jslink((map1, 'zoom'), (map2, 'zoom'))
```
- JavaScript-level linking for smooth interaction
- No Python callback overhead
- Bi-directional synchronization

**Event Handlers:**
- `on_click` for feature selection
- `.observe()` for widget changes
- Layer management (add/remove) for highlights

**State Management:**
- Track current highlights to prevent duplicates
- Store popup references for cleanup
- Maintain active dataset/threshold selections

In [5]:
# | label: fig:multi-maps

# -------------------------------
# Dual Map Controller Class
# -------------------------------

import geopandas as gpd

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np

colors = [
    (0, 0.5, 0),  # Green
    (1, 1, 1),
]  # White
positions = [0, 1]
green_r_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

colors = [
    (1, 1, 1),  # Green
    (0, 0.5, 0),
]  # White
positions = [0, 1]
green_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

# Extract RGB values from the predefined RdBu colormap
cmap = plt.get_cmap("RdBu")

# Create a list of colors by sampling the colormap
colors = [cmap(i) for i in range(cmap.N)]

# Create the custom colormap
custom_rdbu = LinearSegmentedColormap.from_list("custom_rdbu", np.array(colors)[20:-20][::-1])

param_dict = {
    "fraction_coarse": {
        "vmin": 0,
        "vmax": 100,
        "short_label": "FCD",
        "label": "Fraction of coarse-dominated sediment (%)",
        "cmap": custom_rdbu,
        "log_scale": False,
    },
    "path_length_norm": {
        "vmin": 1,
        "vmax": 1000,
        "short_label": "Normalized distance",
        "label": "Normalized path length",
        "cmap": green_r_map,
        "log_scale": True,
    },
    "path_to_no_flow": {
        "vmin": 0,
        "vmax": 20,
        "short_label": "Depth to no flow",
        "label": "Depth to shallowest no-flow unit\n or base surface (m)",
        "cmap": green_map,
        "log_scale": False,
    },
}


# -------------------------------
# Dual Map Visualization
# -------------------------------

# Initialize controller
controller = DualMapController(
    width=map_width,
    height=map_height,
    center=center,
    zoom=zoom,
    subbasins=subbasins,
    subbasin_column="Basin_Su_1",
    rivers_gdf=rivers,
    river_intersections_gdf=river_intersections,
    param_dict=param_dict,
)

# -------------------------------
# Create Raster Overlays
# -------------------------------
# Left map: Fraction Coarse Dominated (static reference)
fcd_params = param_dict["fraction_coarse"]
fcd_ave = ds["fraction_coarse"]
fcd_layer = ImageOverlay(
    url=scalar_to_base64_image(
        fcd_ave,
        cmap=fcd_params["cmap"],
        vmin=fcd_params["vmin"],
        vmax=fcd_params["vmax"],
        log_scale=fcd_params["log_scale"],
    ),
    bounds=leaflet_bounds(fcd_ave),
    opacity=1.0,
    name="Raster Dataset",  # Generic name for layer control
)

# -------------------------------
# Assemble and Display Maps
# -------------------------------
# Add base vector layers to both maps
controller.add_layer(l_rivers)
controller.add_layer(l_subbasins)
controller.add_layer(l_brackish)

# Add scalar raster layers to each map
controller.add_single_scalar_layer(fcd_layer, map_side="left")

# Add multiple scalar layer with dataset selector (right map)
# Specify which datasets should be available in the dropdown
dataset_names = ["path_length_norm", "path_to_no_flow"]
controller.add_multiple_scalar_layer(
    ds,
    dataset_names=dataset_names,
    initial_dataset="path_length_norm",
    initial_threshold=20,
    map_side="right",
)

# Add controls and interactive elements
controller.add_controls()  # Must be called AFTER adding base layers
controller.add_river_intersections(intersections_layer)
controller.create_subbasin_selector(center=center, zoom=zoom)
controller.display()

Output()

Button(button_style='info', description='Home', icon='home', layout=Layout(width='100px'), style=ButtonStyle()…